In [18]:
import transformers as tr
import torch as ts
# use own pytorch training. matching saved with noted model weights is probably easier than. also try using own data set

In [19]:
tokenizer = tr.AutoTokenizer.from_pretrained("bert-base-cased")

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\Ameno/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve

In [20]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(encoded_input)

{'input_ids': tensor([[  101,  1252,  1184,  1164,  1248,  6462,   136,   102,     0,     0,
             0,     0,     0,     0,     0],
        [  101,  1790,   112,   189,  1341,  1119,  3520,  1164,  1248,  6462,
           117, 21902,  1643,   119,   102],
        [  101,  1327,  1164,  5450, 23434,   136,   102,     0,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [21]:
tokenizer.decode(encoded_input["input_ids"][0])

'[CLS] But what about second breakfast? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [22]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

Reusing dataset yelp_review_full (C:\Users\Ameno\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|██████████| 2/2 [00:00<00:00, 18.52it/s]


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [23]:
tokenized_datasets = dataset.map(lambda x: tokenizer(x['text'], padding="max_length", truncation=True), batched=True)

Loading cached processed dataset at C:\Users\Ameno\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-1212de8c9c92504b.arrow
Loading cached processed dataset at C:\Users\Ameno\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-7a0846779351e1d6.arrow


In [24]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(4))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(4))

Loading cached shuffled indices for dataset at C:\Users\Ameno\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-385f4f630def7870.arrow
Loading cached shuffled indices for dataset at C:\Users\Ameno\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-5550b17c557af15e.arrow


In [25]:
small_train_dataset

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4
})

In [26]:
from transformers import AutoModelForSequenceClassification

# Fetches the pre- trained model of BERT, removes the head and replaces it with a classification head with specified output nodes 
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

# Freeze base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\Ameno/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 1

In [27]:
from transformers import TrainingArguments, Trainer


# DO THIS MANUALLY WITH PYTORCH AFTERWARDS 
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=2, per_device_eval_batch_size=2)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    print("Logits,labels:", logits, labels)
    print("Length Logits,labels:", len(logits), len(labels))
    predictions = np.argmax(logits, axis=1)
    print("Predictions:", predictions)
    return metric.compute(predictions=predictions, references=labels)

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset, 
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Ameno\Anaconda3\envs\luna-ai\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 6
 33%|███▎      | 2/6 [00:07<00:15,  3.92s/it]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and ha

Logits,labels: [[ 0.05410584  0.19341856 -0.8401659   0.8545378   0.19235457]
 [-0.05599387  0.30260903 -0.8758489   0.7588924   0.06951551]
 [ 0.03836079  0.12752278 -0.7368164   0.8765589   0.2671244 ]
 [ 0.13608772  0.10909051 -0.6670722   0.9731194   0.32987368]] [2 4 1 4]
Length Logits,labels: 4 4
Predictions: [3 3 3 3]
{'eval_loss': 1.9239552021026611, 'eval_accuracy': 0.0, 'eval_runtime': 7.094, 'eval_samples_per_second': 0.564, 'eval_steps_per_second': 0.282, 'epoch': 1.0}


 67%|██████▋   | 4/6 [00:22<00:11,  5.89s/it]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4
  Batch size = 2
                                             
 67%|██████▋   | 4/6 [00:30<00:11,  5.89s/it]

Logits,labels: [[ 0.06771494  0.16437405 -0.8292247   0.82554805  0.20943274]
 [-0.04230831  0.2731629  -0.8644427   0.7295463   0.08666612]
 [ 0.05318692  0.0970048  -0.72664666  0.84603024  0.28525835]
 [ 0.15064083  0.07893397 -0.65693027  0.94293934  0.3479815 ]] [2 4 1 4]
Length Logits,labels: 4 4
Predictions: [3 3 3 3]
{'eval_loss': 1.9099862575531006, 'eval_accuracy': 0.0, 'eval_runtime': 7.348, 'eval_samples_per_second': 0.544, 'eval_steps_per_second': 0.272, 'epoch': 2.0}


100%|██████████| 6/6 [00:38<00:00,  6.67s/it]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4
  Batch size = 2
                                             
100%|██████████| 6/6 [00:45<00:00,  6.67s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 6/6 [00:45<00:00,  7.63s/it]

Logits,labels: [[ 0.06988452  0.15185758 -0.82136637  0.8130682   0.21830702]
 [-0.04018415  0.26038617 -0.8563896   0.7168367   0.09571047]
 [ 0.05576238  0.08401114 -0.71896327  0.8330267   0.29457402]
 [ 0.15314502  0.06609257 -0.64925736  0.9300949   0.3571967 ]] [2 4 1 4]
Length Logits,labels: 4 4
Predictions: [3 3 3 3]
{'eval_loss': 1.9024713039398193, 'eval_accuracy': 0.0, 'eval_runtime': 7.168, 'eval_samples_per_second': 0.558, 'eval_steps_per_second': 0.279, 'epoch': 3.0}
{'train_runtime': 45.807, 'train_samples_per_second': 0.262, 'train_steps_per_second': 0.131, 'train_loss': 1.8893790245056152, 'epoch': 3.0}


TrainOutput(global_step=6, training_loss=1.8893790245056152, metrics={'train_runtime': 45.807, 'train_samples_per_second': 0.262, 'train_steps_per_second': 0.131, 'train_loss': 1.8893790245056152, 'epoch': 3.0})

In [31]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4
  Batch size = 2
100%|██████████| 2/2 [00:03<00:00,  1.91s/it]

Logits,labels: [[ 0.06988452  0.15185758 -0.82136637  0.8130682   0.21830702]
 [-0.04018415  0.26038617 -0.8563896   0.7168367   0.09571047]
 [ 0.05576238  0.08401114 -0.71896327  0.8330267   0.29457402]
 [ 0.15314502  0.06609257 -0.64925736  0.9300949   0.3571967 ]] [2 4 1 4]
Length Logits,labels: 4 4
Predictions: [3 3 3 3]


{'eval_loss': 1.9024713039398193,
 'eval_accuracy': 0.0,
 'eval_runtime': 7.673,
 'eval_samples_per_second': 0.521,
 'eval_steps_per_second': 0.261,
 'epoch': 3.0}

In [32]:

ts.save(model.state_dict(), "../model/bert.pt")

In [57]:
class hugBert(ts.nn.Module):
  def __init__(self, num_classes):
    super(hugBert, self).__init__()
    # device = ts.device('cuda' if ts.cuda.is_available() else 'cpu')
    self.bert = tr.BertModel.from_pretrained('bert-base-cased', return_dict=True)#.to(device)
    self.classifier = ts.nn.Linear(768, num_classes)#, bias=False)
    # model.classifier = nn.Linear(768, 2) ?
    # can change config number of labels
    # https://discuss.huggingface.co/t/how-do-i-change-the-classification-head-of-a-model/4720/14
  
  # def forward(self, *args, **kwargs):
    
    
  def forward(self, input_ids, token_type_ids, attention_mask, labels):
    # outputs = self.bert(*args, **kwargs)
    print("im here")
    outputs = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

    # print("im herenow", outputs)
    pred = self.classifier(outputs[1])#outputs.pooler_output)
    # pred = self.classifier(outputs[0])#outputs.pooler_output)
    print("pred:",pred)
    # When the input gets labels for the evaluator, it expects the result of this 
    # to be a dictionary with a loss, or an array which has the loss as the first element
    dick = {"results": pred, "loss": ts.rand(len(pred))}
    return dick
    # return pred

# device = ts.device('cuda' if ts.cuda.is_available() else 'cpu')
myModel = hugBert(5)#.to(device)


loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\Ameno/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/main/pytorch_model.bin from c

In [44]:
myModel.load_state_dict(ts.load("../model/bert.pt"))

<All keys matched successfully>

In [58]:
myModel.eval()
testing_args = TrainingArguments(output_dir="test_trainer1", per_device_eval_batch_size=2)#, evaluation_strategy="epoch")
evaluator = Trainer(
    model=myModel,
    args=testing_args,
    # train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset, 
    compute_metrics=compute_metrics,
)
evaluator.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set  don't have a corresponding argument in `hugBert.forward` and have been ignored: text. If text are not expected by `hugBert.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4
  Batch size = 2


im here
pred: tensor([[ 0.2339, -0.2233, -0.5350, -0.4612,  0.1837],
        [ 0.3383, -0.2116, -0.6041, -0.5893,  0.2163]])


im here




100%|██████████| 2/2 [00:02<00:00,  1.37s/it]

pred: tensor([[ 0.3623, -0.3273, -0.7025, -0.5020,  0.1309],
        [ 0.2426, -0.4277, -0.6834, -0.4844,  0.1469]])
Logits,labels: [[ 0.23386702 -0.22330618 -0.53495735 -0.4611584   0.18370311]
 [ 0.3382607  -0.21159908 -0.6040662  -0.58929867  0.21625488]
 [ 0.36230123 -0.3273003  -0.7025295  -0.5019569   0.13094695]
 [ 0.24255146 -0.42766267 -0.6833956  -0.4843667   0.14691721]] [2 4 1 4]
Length Logits,labels: 4 4
Predictions: [0 0 0 0]


{'eval_loss': 0.583378255367279,
 'eval_accuracy': 0.0,
 'eval_runtime': 5.432,
 'eval_samples_per_second': 0.736,
 'eval_steps_per_second': 0.368}

In [ ]:
# test
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4
  Batch size = 2
100%|██████████| 2/2 [00:03<00:00,  1.87s/it]

Logits,labels: [[ 0.06988452  0.15185758 -0.82136637  0.8130682   0.21830702]
 [-0.04018415  0.26038617 -0.8563896   0.7168367   0.09571047]
 [ 0.05576238  0.08401114 -0.71896327  0.8330267   0.29457402]
 [ 0.15314502  0.06609257 -0.64925736  0.9300949   0.3571967 ]] [2 4 1 4]
Length Logits,labels: 4 4
Predictions: [3 3 3 3]


{'eval_loss': 1.9024713039398193,
 'eval_accuracy': 0.0,
 'eval_runtime': 7.3045,
 'eval_samples_per_second': 0.548,
 'eval_steps_per_second': 0.274,
 'epoch': 3.0}